In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
from fibsem import utils, acquire, movement, calibration
from fibsem.structures import ImageSettings, BeamType
from pprint import pprint
import matplotlib.pyplot as plt
from fibsem import milling
import numpy as np


In [4]:
protocol_path = r"C:\Users\Admin\Github\fibsem\example\protocol\protocol.yaml"
microscope, settings = utils.setup_session(protocol_path=protocol_path, session_path= r"D:\cyril\slice-and-view-cyril_2022-10-14.02-18-00PM")

2022-10-14 16:12:12,400 — root — INFO — connect_to_microscope:31 — Microscope client connecting to [10.0.0.1]
2022-10-14 16:12:12,408 — root — INFO — connect_to_microscope:34 — Microscope client connected to [10.0.0.1]
2022-10-14 16:12:12,409 — root — INFO — setup_session:252 — Finished setup for session: slice-and-view-cyril_2022-10-14.04-12-12PM


Client connecting to [10.0.0.1:7520]...
Client connected to [10.0.0.1:7520]


In [5]:
settings.image.save_path

# settings.image.save = True
# eb_image = acquire.new_image(microscope, settings.image)
# eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)


'D:\\cyril\\slice-and-view-cyril_2022-10-14.02-18-00PM'

In [ ]:
# settings
# microscope.imaging.set_active_device(BeamType.ELECTRON.value)
# microscope.imaging.set_active_view(BeamType.ELECTRON.value)
# microscope.beams.electron_beam.optical_mode.value = "Immersion"
# microscope.detector.mode.value = "BackscatterElectrons"
# settings.system.stage.tilt_flat_to_electron = 0
# import numpy as np
# microscope.beams.electron_beam.angular_correction.angle.value = np.deg2rad(-38)

# # LUT: inverse


In [ ]:
from autoscript_sdb_microscope_client.enumerations import DetectorMode, DetectorType
# microscope.detector.set_type_mode(type=DetectorType.ETD, mode = DetectorMode.BACKSCATTER_ELECTRONS)

In [7]:
from fibsem.structures import MicroscopeSettings
from autoscript_sdb_microscope_client import SdbMicroscopeClient


def run_data_collection(microscope: SdbMicroscopeClient, settings: MicroscopeSettings, dwell_times: list[float], step: int):
    
    settings.image.save = True
    settings.image.autocontrast = False

    for i, dwell_time in enumerate(dwell_times):

        settings.image.dwell_time = dwell_time
        print(f"{i} - dwell_time: {dwell_time}")

        # take multiple short dwell time images
        if dwell_time < 2e-6:
            for j in range(7):
                settings.image.label = f"EB_{step}_{dwell_time}_{j}"
                print(i, j, dwell_time, settings.image.label)
                acquire.new_image(microscope, settings.image)
        else:
            settings.image.label = f"EB_{step}_{dwell_time}_0"
            print(i, 0, dwell_time, settings.image.label)
            # take a single high dwell time
            acquire.new_image(microscope, settings.image)
        
        print("-"*50)

In [8]:
# settings.image.save_path = r"D:\cyril\slice-and-view-cyril_2022-10-14.02-18-00PM"
print(settings.image.save_path)

D:\cyril\slice-and-view-cyril_2022-10-14.02-18-00PM


In [9]:


# start wiith cleaning cross section to clean the area
centre_x = 0
centre_y = 0
width = 40e-6
height = 2.5e-6
depth = 10e-6

microscope.imaging.set_active_view(BeamType.ION.value)
microscope.imaging.set_active_device(BeamType.ION.value)
microscope.patterning.clear_patterns()

pattern = microscope.patterning.create_cleaning_cross_section(center_x=centre_x, center_y=centre_y, width=width, height=height, depth=depth)
pattern.scan_direction = "BottomToTop"

In [8]:
# settings
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.imaging.set_active_view(BeamType.ELECTRON.value)
microscope.detector.contrast.limits

Limits(min=0, max=1)

In [11]:
state = calibration.get_current_microscope_state(microscope)
# detector contrast brightness
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.imaging.set_active_view(BeamType.ELECTRON.value)
contrast = microscope.detector.contrast.value
brightness = microscope.detector.brightness.value

In [14]:
# microscope.beams.electron_beam.beam_current.value = 0.1e-9
print(brightness, contrast)

0.3743122542104864 0.8333920280603576


In [ ]:
acquire.autocontrast(microscope, BeamType.ELECTRON)

In [12]:
start_x, end_x = -8e-6, 33e-6
start_y, end_y = -4e-6, -4e-6
depth = 8e-6
offset = 5e-9


N_SLICES = 500
START_IDX = 22
# for i in range(START_IDX, N_SLICES):

microscope.imaging.set_active_view(BeamType.ION.value)
microscope.imaging.set_active_device(BeamType.ION.value)
microscope.patterning.clear_patterns()
pattern = microscope.patterning.create_line(start_x=start_x, start_y=start_y, end_x=end_x, end_y=end_y, depth=depth)
print(pattern)

In [13]:

settings.image.hfw = 30e-6
settings.image.resolution = "6144x4096"
dwell_times = [0.5e-6, 1e-6, 3e-6, 5e-6, 8e-6, 20e-6]
settings.image.save = True
settings.image.autocontrast = False
settings.image.gamma.enabled = False

start_x, end_x = -8e-6, 33e-6
start_y, end_y = -4e-6, -4e-6
depth = 8e-6
offset = 5e-9

N_SLICES = 500
START_IDX = 0
for i in range(START_IDX, N_SLICES):

    print(f" ---------------- SLICE {i}/{N_SLICES} ---------------- ")
    
    settings.image.dwell_time = 0.2e-6
    settings.image.save = False
    # slice
    if i > START_IDX:
        microscope.imaging.set_active_view(BeamType.ION.value)
        microscope.imaging.set_active_device(BeamType.ION.value)
        microscope.patterning.clear_patterns()
        pattern = microscope.patterning.create_line(start_x=start_x, start_y=start_y, end_x=end_x, end_y=end_y, depth=depth)
        print(pattern)

        # run
        milling.run_milling(microscope, milling_current=5.6e-9)
    
    # view
    calibration.auto_discharge_beam(microscope, settings.image, n_iterations=5)

    settings.image.autocontrast = False
    run_data_collection(microscope, settings, dwell_times, i)

    start_y += offset
    end_y  += offset
    
    # manually adjust working distance
    wd_diff = offset * np.sin(np.deg2rad(38))
    microscope.beams.electron_beam.working_distance.value -= wd_diff #4e-3# 3.995e-3 

    if i % 50 == 0 and i != START_IDX:
        acquire.autocontrast(microscope, BeamType.ELECTRON)


2022-10-14 16:15:14,440 — root — INFO — auto_discharge_beam:141 — Bring me Thanos!
2022-10-14 16:15:14,444 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


 ---------------- SLICE 0/500 ---------------- 


2022-10-14 16:15:16,670 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.
2022-10-14 16:15:17,152 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.
2022-10-14 16:15:17,601 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.
2022-10-14 16:15:18,036 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.
2022-10-14 16:15:18,474 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.
2022-10-14 16:15:25,079 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


0 - dwell_time: 5e-07
0 0 5e-07 EB_0_5e-07_0


2022-10-14 16:15:38,980 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


...

--------------------------------------------------
3 - dwell_time: 5e-06
3 0 5e-06 EB_0_5e-06_0


2022-10-14 16:23:33,934 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


--------------------------------------------------
4 - dwell_time: 8e-06
4 0 8e-06 EB_0_8e-06_0


2022-10-14 16:26:58,919 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


--------------------------------------------------
5 - dwell_time: 2e-05
5 0 2e-05 EB_0_2e-05_0


In [ ]:
# microscope.auto_functions.run_auto_focus()
from autoscript_sdb_microscope_client.structures import RunAutoFocusSettings

focus_settings = RunAutoFocusSettings()

microscope.imaging.set_active_view(BeamType.ELECTRON.value)
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.auto_functions.run_auto_focus()
microscope.specimen.stage.link()


# microscope.auto_functions.run_auto_stigmator()

In [ ]:
settings.image.hfw = 30e-6
settings.image.resolution = "6144x4096"
dwell_times = [0.3e-6, 0.5e-6, 1e-6, 3e-6, 5e-6, 8e-6]
settings.image.save = True
settings.image.autocontrast = True
settings.image.gamma.enabled = False
settings.image.dwell_time = 0.5e-6
settings.image.label = "4um"
microscope.beams.electron_beam.working_distance.value = 4e-3# 3.995e-3 
acquire.new_image(microscope, settings.image)

In [ ]:
# # dynamic focus : 38



# 4e-3 - (1000*5e-9)





In [ ]:
settings.image.save_path

In [ ]:
# multichem / GIS query